<a href="https://colab.research.google.com/github/peace-and-harmony/image-matting/blob/main/notebooks/MODNet_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#MODNet -clothing matting model evaluatiaon

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!nvidia-smi

Thu Sep 23 15:01:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Installation

In [ ]:
%%capture
!git clone https://github.com/peace-and-harmony/image-matting.git
!git clone https://github.com/ZHKKKe/MODNet.git
!pip install --upgrade pillow

# kaggle clothing dataset

- https://www.kaggle.com/agrigorev/clothing-dataset-full

Find your kaggle username and kaggle key to download the clothing dataset for model test

In [ ]:
#Configuration environment
import os

os.environ['KAGGLE_USERNAME'] = "XXXXXX" # username from the json file
os.environ['KAGGLE_KEY'] = "XXXXXX" # key from the json file

In [ ]:
!kaggle datasets download -d agrigorev/clothing-dataset-full

100% 6.49G/6.50G [02:21<00:00, 23.0MB/s]
100% 6.50G/6.50G [02:21<00:00, 49.3MB/s]


In [ ]:
# %%capture
!unzip clothing-dataset-full.zip

In [ ]:
import glob
len(glob.glob('/content/images_compressed/*.jpg'))

5762

# Data preparation

In [ ]:
%cd /content/
%ls

/content
drive/  image-matting/  MODNet/  sample_data/


In [ ]:
%cd /content
import glob 
import shutil
import shutil, random, os

clothing_alt = True #@param {type:"boolean"}
eval_set = True #@param {type:"boolean"}

if clothing_alt:
  # test without ground truth
  path = '/content/images_compressed'
  dest = '/content/images_compressed_random_pick'

  if not os.path.isdir(dest):
    os.makedirs(dest)

  images = random.sample(glob.glob(f'{path}/*'), 200)
  for f in images:
    shutil.copy(f, dest)


if eval_set:
  # model MIoU evaluation
  # Load the validation data from gdrive
  !cp /content/drive/MyDrive/XXXX/validation_data.zip /content/
  !unzip validation_data


# Evaluation

In [ ]:
%cd /content/
!cp /content/image-matting/scripts/modnet_eval.py .

/content


## Load checkpoint

In [ ]:
%ls /content/image-matting/pretrained_weights

checkpoint.pth


In [ ]:
import torch

if torch.cuda.is_available():
  device = torch.device('cuda')
  print('using gpu!')
else:
  device = torch.device('cpu')

weights_path = '/content/image-matting/pretrained_weights/checkpoint.pth'
 
check = torch.load('/content/image-matting/pretrained_weights/checkpoint.pth', map_location=device)
torch.save(check['state_dict'], '/content/checkpoint.pth')
print('Checkpoint Loaded!')

using gpu!
Checkpoint Loaded!


## Inference

In [ ]:
%cd /content

import os
import shutil

clothing_alt = True

if clothing_alt:
  eval_set = False
else:
  eval_set = True
test_out = '/content/test/'

if os.path.isdir(test_out):
  shutil.rmtree(test_out)
if clothing_alt:
  !python modnet_eval.py --images /content/images_compressed_random_pick/ --model /content/checkpoint.pth --out {test_out}
else:
  !python modnet_eval.py --images /content/validation_data/image --labels /content/validation_data/mask --model /content/checkpoint.pth --out {test_out}

In [ ]:
from IPython.display import display,  clear_output
from ipywidgets import Dropdown, Output
 
import cv2
import glob
import os
import numpy as np
from google.colab.patches import cv2_imshow

img_path = '/content/test'

images = glob.glob(f'{img_path}/*')
out = Output()
 
def dropdown_eventhandler(change):
    with out:
        clear_output()
        print('loading...')
        img = cv2.imread(change.new, cv2.IMREAD_UNCHANGED)
        img = cv2.resize(img, (0, 0), fx=1., fy=1.0) 
        cv2_imshow(img)

dropdown = Dropdown(description="Choose one:", options=images)
dropdown.observe(dropdown_eventhandler, names='value')
display(dropdown)
display(out)

Dropdown(description='Choose one:', options=('/content/test/bb8a413c-c2fa-4b2f-9ce9-f6e44cbf0ecd.png', '/conte…

Output()